### Model Training

Once your model is defined, it can be trained. This can happen on a single thread or on a parallel framework like Watson Machine Learning or Apache Spark. In the most simple case Model Definition and Model Training is just a couple of LOCs (lines of code) away. In the case of Watson Machine Learning or Apache Spark models might need to get serialized and transferred to another technology / framework.

Please specify and justify the technologies used for model definition and training in the ADD. 

Once you think you have achieved a descent model performance save the notebook according to the process model’s naming convention and proceed to the model evaluation task.

In [12]:
!pip install tensorflow
!pip install python-mnist
!pip install Pillow
!pip install pyspark

  Using cached tensorflow-2.5.0-cp37-cp37m-manylinux2010_x86_64.whl (454.3 MB)
  Using cached wheel-0.36.2-py2.py3-none-any.whl (35 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
Processing ./.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6/wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached keras_nightly-2.5.0.dev2021032900-py2.py3-none-any.whl (1.2 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached absl_py-0.13.0-py3-none-any.whl (132 kB)
  Using cached numpy-1.19.5-cp37-cp37m-manylinux2010_x86_64.whl (14.8 MB)
  Using cached tensorboard-2.5.0-py3-none-any.whl (6.0 MB)
  Using cached tensorflow_estimator-2.5.0-py2.py3-none-any.whl (462 kB)
  Using cached protobuf-3.17.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.0 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)

In [13]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import image
import tensorflow as tf
import seaborn as sns
from mnist import MNIST
import numpy as np
import PIL
from PIL import Image
import os
import matplotlib.image as mping
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer
from numpy import asarray
import pandas as pd
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout
from tensorflow.keras import backend as K
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA

tf.__version__

'2.5.0'

In [14]:
PIL.__version__

'8.2.0'

In [15]:
# fire up the spark session
# comment this cell out for a spark server
#sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

#spark = SparkSession \
#    .builder \
#    .getOrCreate()

In [16]:
# enable arrow which lets us transfrom a pandas dataframe into a pyspark dataframe
spark.conf.set("spark.sql.execution.arrow.enabled","true")

## First dataset

#### 60,000 images and 10 classes, each image is 28 x 28 or represented by a 794 element array

In [6]:
!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/train-images-idx3-ubyte.gz?raw=True
!mv train-images-idx3-ubyte.gz?raw=True train-images-idx3-ubyte.gz
!gunzip train-images-idx3-ubyte.gz
!ls -lahr train-images-idx3-ubyte

!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/train-labels-idx1-ubyte.gz?raw=True
!mv train-labels-idx1-ubyte.gz?raw=True train-labels-idx1-ubyte.gz
!gunzip train-labels-idx1-ubyte.gz
!ls -lahr train-labels-idx1-ubyte

!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/t10k-images-idx3-ubyte.gz?raw=True
!mv t10k-images-idx3-ubyte.gz?raw=True t10k-images-idx3-ubyte.gz
!gunzip t10k-images-idx3-ubyte.gz
!ls -lahr t10k-images-idx3-ubyte

!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/t10k-labels-idx1-ubyte.gz?raw=True
!mv t10k-labels-idx1-ubyte.gz?raw=True t10k-labels-idx1-ubyte.gz
!gunzip t10k-labels-idx1-ubyte.gz
!ls -lahr t10k-labels-idx1-ubyte

--2021-06-21 19:33:52--  http://codh.rois.ac.jp/kmnist/dataset/kmnist/train-images-idx3-ubyte.gz?raw=True
Resolving codh.rois.ac.jp (codh.rois.ac.jp)... 136.187.88.58
Connecting to codh.rois.ac.jp (codh.rois.ac.jp)|136.187.88.58|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18165135 (17M)
Saving to: ‘train-images-idx3-ubyte.gz?raw=True’

train-images-idx3-u 100%[===================>]  17.32M  4.99MB/s    in 3.5s    

2021-06-21 19:33:56 (4.99 MB/s) - ‘train-images-idx3-ubyte.gz?raw=True’ saved [18165135/18165135]

-rw-rw-r-- 1 spark spark 45M Feb  4  2019 train-images-idx3-ubyte
--2021-06-21 19:33:59--  http://codh.rois.ac.jp/kmnist/dataset/kmnist/train-labels-idx1-ubyte.gz?raw=True
Resolving codh.rois.ac.jp (codh.rois.ac.jp)... 136.187.88.58
Connecting to codh.rois.ac.jp (codh.rois.ac.jp)|136.187.88.58|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29497 (29K)
Saving to: ‘train-labels-idx1-ubyte.gz?raw=True’

train-labels-idx1-u 100

In [7]:
!mkdir kmnistdata
!cp t10k-images-idx3-ubyte kmnistdata/t10k-images-idx3-ubyte
!cp t10k-labels-idx1-ubyte kmnistdata/t10k-labels-idx1-ubyte
!cp train-images-idx3-ubyte kmnistdata/train-images-idx3-ubyte
!cp train-labels-idx1-ubyte kmnistdata/train-labels-idx1-ubyte
!ls -al kmnistdata

data = MNIST('kmnistdata')
train_images, train_labels = data.load_training()
test_images, test_labels = data.load_testing()

train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

train_images = train_images / 255
test_images = test_images / 255

total 53908
drwxrwxr-x  2 spark spark     4096 Jun 21 19:34 .
drwxr-xr-x 11 spark  2000     4096 Jun 21 19:34 ..
-rw-rw-r--  1 spark spark  7840016 Jun 21 19:34 t10k-images-idx3-ubyte
-rw-rw-r--  1 spark spark    10008 Jun 21 19:34 t10k-labels-idx1-ubyte
-rw-rw-r--  1 spark spark 47040016 Jun 21 19:34 train-images-idx3-ubyte
-rw-rw-r--  1 spark spark    60008 Jun 21 19:34 train-labels-idx1-ubyte


In [8]:
# model_1 def

model_1 = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [9]:
# model_1 compile
model_1.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
# model_1 fit

# began by using 5 epochs here but observed that 
# the validation accuracy is increasing even as we
# stopped training, so consider using more epochs here next

model_1.fit(train_images, train_labels, epochs=5, verbose=1, validation_data=(test_images, test_labels))

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4048 - accuracy: 0.8803 - val_loss: 0.5066 - val_accuracy: 0.8456
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1947 - accuracy: 0.9426 - val_loss: 0.4295 - val_accuracy: 0.8718
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1385 - accuracy: 0.9597 - val_loss: 0.3975 - val_accuracy: 0.8907
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1047 - accuracy: 0.9696 - val_loss: 0.4275 - val_accuracy: 0.8801
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0809 - accuracy: 0.9761 - val_loss: 0.3971 - val_accuracy: 0.8982


In [11]:
score = model_1.evaluate(test_images, test_labels, verbose=0)
print('model_1 test loss:', score[0])
print('model_1 test accuracy', score[1])

model_1 test loss: 0.3971461057662964
model_1 test accuracy 0.8981999754905701


In [38]:
# model_1_1 def

model_1_1 = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),    
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [39]:
# model_1_1 compile
model_1_1.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [40]:
# model_1_1 fit
model_1_1.fit(train_images, train_labels, epochs=5, verbose=1, validation_data=(test_images, test_labels))

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4807 - accuracy: 0.8521 - val_loss: 0.5372 - val_accuracy: 0.8344
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2692 - accuracy: 0.9171 - val_loss: 0.4496 - val_accuracy: 0.8587
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2181 - accuracy: 0.9329 - val_loss: 0.4092 - val_accuracy: 0.8790
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1801 - accuracy: 0.9444 - val_loss: 0.3915 - val_accuracy: 0.8871
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1662 - accuracy: 0.9484 - val_loss: 0.3839 - val_accuracy: 0.8890


In [41]:
model_1_1.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_33 (Dense)             (None, 128)               100480    
_________________________________________________________________
dropout_9 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_34 (Dense)             (None, 128)               16512     
_________________________________________________________________
dropout_10 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_35 (Dense)             (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


In [52]:
# model_1_2 def

model_1_2 = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),    
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),    
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [53]:
# model_1_2 compile
model_1_2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [54]:
# model_1_2 fit
model_1_2.fit(train_images, train_labels, epochs=20, verbose=1, validation_data=(test_images, test_labels))

Epoch 1/20
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3722 - accuracy: 0.8857 - val_loss: 0.5198 - val_accuracy: 0.8383
Epoch 2/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1791 - accuracy: 0.9460 - val_loss: 0.4599 - val_accuracy: 0.8676
Epoch 3/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1261 - accuracy: 0.9617 - val_loss: 0.4123 - val_accuracy: 0.8814
Epoch 4/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1046 - accuracy: 0.9681 - val_loss: 0.4151 - val_accuracy: 0.8933
Epoch 5/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0839 - accuracy: 0.9747 - val_loss: 0.4230 - val_accuracy: 0.8908
Epoch 6/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0675 - accuracy: 0.9791 - val_loss: 0.4180 - val_accuracy: 0.8970
Epoch 7/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0614 - accuracy: 0.9805 - val_loss: 0.4154 - val_accuracy:

In [45]:
model_1_2.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_36 (Dense)             (None, 128)               100480    
_________________________________________________________________
dense_37 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_38 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_39 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_40 (Dense)             (None, 10)                1290      
Total params: 151,306
Trainable params: 151,306
Non-trainable params: 0
_________________________________________________________________


In [49]:
# model_1_3 def

model_1_3 = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),    
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),    
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [50]:
# model_1_3 compile
model_1_3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [51]:
# model_1_3 fit
model_1_3.fit(train_images, train_labels, epochs=20, verbose=1, validation_data=(test_images, test_labels))

Epoch 1/20
1875/1875 [==============================] - 5s 2ms/step - loss: 0.4513 - accuracy: 0.8613 - val_loss: 0.5261 - val_accuracy: 0.8399
Epoch 2/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2188 - accuracy: 0.9348 - val_loss: 0.4477 - val_accuracy: 0.8650
Epoch 3/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1648 - accuracy: 0.9506 - val_loss: 0.4268 - val_accuracy: 0.8837
Epoch 4/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1326 - accuracy: 0.9604 - val_loss: 0.3916 - val_accuracy: 0.8929
Epoch 5/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1138 - accuracy: 0.9666 - val_loss: 0.3983 - val_accuracy: 0.8895
Epoch 6/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0943 - accuracy: 0.9714 - val_loss: 0.4388 - val_accuracy: 0.8899
Epoch 7/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0860 - accuracy: 0.9750 - val_loss: 0.4374 - val_accuracy:

In [55]:
# model_1_4 def

model_1_4 = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),    
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),    
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [56]:
# model_1_4 compile

model_1_4.compile(optimizer=tf.keras.optimizers.Adadelta(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [57]:
# model_1_4 fit

model_1_4.fit(train_images, train_labels, epochs=20, verbose=1, validation_data=(test_images, test_labels))

Epoch 1/20
1875/1875 [==============================] - 5s 3ms/step - loss: 2.2887 - accuracy: 0.1389 - val_loss: 2.2826 - val_accuracy: 0.1397
Epoch 2/20
1875/1875 [==============================] - 4s 2ms/step - loss: 2.2387 - accuracy: 0.2098 - val_loss: 2.2480 - val_accuracy: 0.1768
Epoch 3/20
1875/1875 [==============================] - 4s 2ms/step - loss: 2.1765 - accuracy: 0.2908 - val_loss: 2.2032 - val_accuracy: 0.2277
Epoch 4/20
1875/1875 [==============================] - 4s 2ms/step - loss: 2.0938 - accuracy: 0.3853 - val_loss: 2.1450 - val_accuracy: 0.2882
Epoch 5/20
1875/1875 [==============================] - 4s 2ms/step - loss: 1.9884 - accuracy: 0.4529 - val_loss: 2.0727 - val_accuracy: 0.3376
Epoch 6/20
1875/1875 [==============================] - 4s 2ms/step - loss: 1.8645 - accuracy: 0.5017 - val_loss: 1.9919 - val_accuracy: 0.3773
Epoch 7/20
1875/1875 [==============================] - 4s 2ms/step - loss: 1.7342 - accuracy: 0.5435 - val_loss: 1.9111 - val_accuracy:

In [8]:
# model_1_4_1 def (using the dropouts with adadelta)

model_1_4_1 = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),    
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [9]:
# model_1_4_1 compile

model_1_4_1.compile(optimizer=tf.keras.optimizers.Adadelta(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
# model_1_4_1 fit

model_1_4_1.fit(train_images, train_labels, epochs=20, verbose=1, validation_data=(test_images, test_labels))

Epoch 1/20
1875/1875 [==============================] - 11s 6ms/step - loss: 2.3343 - accuracy: 0.1264 - val_loss: 2.2810 - val_accuracy: 0.1523
Epoch 2/20
1875/1875 [==============================] - 10s 5ms/step - loss: 2.2410 - accuracy: 0.1822 - val_loss: 2.2179 - val_accuracy: 0.2198
Epoch 3/20
1875/1875 [==============================] - 10s 5ms/step - loss: 2.1556 - accuracy: 0.2451 - val_loss: 2.1599 - val_accuracy: 0.2808
Epoch 4/20
1875/1875 [==============================] - 10s 5ms/step - loss: 2.0734 - accuracy: 0.3065 - val_loss: 2.1044 - val_accuracy: 0.3320
Epoch 5/20
1875/1875 [==============================] - 10s 5ms/step - loss: 1.9978 - accuracy: 0.3560 - val_loss: 2.0492 - val_accuracy: 0.3700
Epoch 6/20
1875/1875 [==============================] - 10s 5ms/step - loss: 1.9217 - accuracy: 0.3990 - val_loss: 1.9935 - val_accuracy: 0.3994
Epoch 7/20
1875/1875 [==============================] - 10s 5ms/step - loss: 1.8444 - accuracy: 0.4379 - val_loss: 1.9381 - val_ac

In [5]:
# model_1_5 def

model_1_5 = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),    
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),    
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [6]:
# model_1_5 compile

model_1_5.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
# model_1_5 fit

model_1_5.fit(train_images, train_labels, epochs=20, verbose=1, validation_data=(test_images, test_labels))

Epoch 1/20
1875/1875 [==============================] - 5s 2ms/step - loss: 0.9446 - accuracy: 0.7149 - val_loss: 0.8656 - val_accuracy: 0.7227
Epoch 2/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4174 - accuracy: 0.8721 - val_loss: 0.6876 - val_accuracy: 0.7840
Epoch 3/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3243 - accuracy: 0.9013 - val_loss: 0.6232 - val_accuracy: 0.8074
Epoch 4/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2668 - accuracy: 0.9203 - val_loss: 0.5939 - val_accuracy: 0.8173
Epoch 5/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2259 - accuracy: 0.9320 - val_loss: 0.5436 - val_accuracy: 0.8390
Epoch 6/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1958 - accuracy: 0.9427 - val_loss: 0.5060 - val_accuracy: 0.8485
Epoch 7/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1698 - accuracy: 0.9505 - val_loss: 0.4889 - val_accuracy:

In [17]:
# model_1_5_1 def

model_1_5_1 = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),    
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [18]:
# model_1_5_1 compile

model_1_5_1.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
# model_1_5_1 fit

model_1_5_1.fit(train_images, train_labels, epochs=20, verbose=1, validation_data=(test_images, test_labels))

Epoch 1/20
1875/1875 [==============================] - 9s 5ms/step - loss: 1.0603 - accuracy: 0.6719 - val_loss: 0.9947 - val_accuracy: 0.6860
Epoch 2/20
1875/1875 [==============================] - 8s 4ms/step - loss: 0.6080 - accuracy: 0.8130 - val_loss: 0.8254 - val_accuracy: 0.7384
Epoch 3/20
1875/1875 [==============================] - 8s 4ms/step - loss: 0.4905 - accuracy: 0.8513 - val_loss: 0.7144 - val_accuracy: 0.7748
Epoch 4/20
1875/1875 [==============================] - 8s 4ms/step - loss: 0.4224 - accuracy: 0.8715 - val_loss: 0.6567 - val_accuracy: 0.7926
Epoch 5/20
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3763 - accuracy: 0.8852 - val_loss: 0.6121 - val_accuracy: 0.8071
Epoch 6/20
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3469 - accuracy: 0.8936 - val_loss: 0.5718 - val_accuracy: 0.8230
Epoch 7/20
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3212 - accuracy: 0.9036 - val_loss: 0.5500 - val_accuracy:

In [11]:
# model_2 def

# optimizer is adam and loss is sparse_categorical_crossentropy

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

x_train = train_images
x_test = test_images
y_train = train_labels
y_test = test_labels

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

model_2 = tf.keras.Sequential()
model_2.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model_2.add(Conv2D(64, (3, 3), activation='relu'))
model_2.add(MaxPooling2D(pool_size=(2, 2)))
model_2.add(Dropout(0.25))
model_2.add(tf.keras.layers.Flatten())
model_2.add(tf.keras.layers.Dense(128, activation='relu'))
model_2.add(Dropout(0.5))
model_2.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

# model_2 compile

model_2.compile(optimizer='adam', 
          loss='sparse_categorical_crossentropy',
          metrics=['accuracy'])

In [ ]:
# fit model

model_2.fit(x_train, y_train,
         batch_size=batch_size,
         epochs=epochs,
         verbose=1,
         validation_data=(x_test, y_test))

Epoch 1/12
469/469 [==============================] - 211s 450ms/step - loss: 0.4129 - accuracy: 0.8742 - val_loss: 0.3417 - val_accuracy: 0.9001
Epoch 2/12
469/469 [==============================] - 209s 445ms/step - loss: 0.1737 - accuracy: 0.9475 - val_loss: 0.2736 - val_accuracy: 0.9198
Epoch 3/12
469/469 [==============================] - 208s 443ms/step - loss: 0.1290 - accuracy: 0.9598 - val_loss: 0.2315 - val_accuracy: 0.9371
Epoch 4/12
469/469 [==============================] - 209s 446ms/step - loss: 0.1016 - accuracy: 0.9682 - val_loss: 0.2087 - val_accuracy: 0.9409
Epoch 5/12
469/469 [==============================] - 208s 443ms/step - loss: 0.0820 - accuracy: 0.9740 - val_loss: 0.2093 - val_accuracy: 0.9436
Epoch 6/12
469/469 [==============================] - 208s 443ms/step - loss: 0.0701 - accuracy: 0.9777 - val_loss: 0.2163 - val_accuracy: 0.9427
Epoch 7/12
469/469 [==============================] - 208s 443ms/step - loss: 0.0663 - accuracy: 0.9783 - val_loss: 0.2089 -

In [ ]:
# model_2_1: change of optimizer to Adadelta and compile

model_2_1 = tf.keras.Sequential()
model_2_1.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model_2_1.add(Conv2D(64, (3, 3), activation='relu'))
model_2_1.add(MaxPooling2D(pool_size=(2, 2)))
model_2_1.add(Dropout(0.25))
model_2_1.add(tf.keras.layers.Flatten())
model_2_1.add(tf.keras.layers.Dense(128, activation='relu'))
model_2_1.add(Dropout(0.5))
model_2_1.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

model_2_1.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adadelta(learning_rate=1.0),
              metrics=['accuracy'])

In [ ]:
model_2_1.fit(x_train, y_train,
           batch_size=batch_size,
           epochs=epochs,
           verbose=1,
           validation_data=(x_test, y_test))

In [11]:
# optimizer = Adamax

# model_2_2: change of optimizer to Adamax and compile

model_2_2 = tf.keras.Sequential()
model_2_2.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model_2_2.add(Conv2D(64, (3, 3), activation='relu'))
model_2_2.add(MaxPooling2D(pool_size=(2, 2)))
model_2_2.add(Dropout(0.25))
model_2_2.add(tf.keras.layers.Flatten())
model_2_2.add(tf.keras.layers.Dense(128, activation='relu'))
model_2_2.add(Dropout(0.5))
model_2_2.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

model_2_2.compile(loss='sparse_categorical_crossentropy',
              optimizer='Adamax',
              metrics=['accuracy'])

In [12]:
model_2_2.fit(x_train, y_train,
           batch_size=batch_size,
           epochs=epochs,
           verbose=1,
           validation_data=(x_test, y_test))

Epoch 1/12
469/469 [==============================] - 95s 202ms/step - loss: 0.5973 - accuracy: 0.8165 - val_loss: 0.5072 - val_accuracy: 0.8421
Epoch 2/12
469/469 [==============================] - 93s 199ms/step - loss: 0.2788 - accuracy: 0.9162 - val_loss: 0.3876 - val_accuracy: 0.8847
Epoch 3/12
469/469 [==============================] - 94s 200ms/step - loss: 0.2121 - accuracy: 0.9376 - val_loss: 0.3238 - val_accuracy: 0.9056
Epoch 4/12
469/469 [==============================] - 93s 199ms/step - loss: 0.1723 - accuracy: 0.9477 - val_loss: 0.3036 - val_accuracy: 0.9139
Epoch 5/12
469/469 [==============================] - 94s 200ms/step - loss: 0.1514 - accuracy: 0.9544 - val_loss: 0.2632 - val_accuracy: 0.9246
Epoch 6/12
469/469 [==============================] - 94s 200ms/step - loss: 0.1321 - accuracy: 0.9598 - val_loss: 0.2511 - val_accuracy: 0.9304
Epoch 7/12
469/469 [==============================] - 93s 198ms/step - loss: 0.1150 - accuracy: 0.9647 - val_loss: 0.2460 - val_ac

In [14]:
# optimizer = Adagrad

# model_2_3: change of optimizer to Adagrad and compile

model_2_3 = tf.keras.Sequential()
model_2_3.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model_2_3.add(Conv2D(64, (3, 3), activation='relu'))
model_2_3.add(MaxPooling2D(pool_size=(2, 2)))
model_2_3.add(Dropout(0.25))
model_2_3.add(tf.keras.layers.Flatten())
model_2_3.add(tf.keras.layers.Dense(128, activation='relu'))
model_2_3.add(Dropout(0.5))
model_2_3.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

model_2_3.compile(loss='sparse_categorical_crossentropy',
              optimizer='Adagrad',
              metrics=['accuracy'])

In [15]:
model_2_3.fit(x_train, y_train,
           batch_size=batch_size,
           epochs=epochs,
           verbose=1,
           validation_data=(x_test, y_test))

Epoch 1/12
469/469 [==============================] - 214s 455ms/step - loss: 2.0159 - accuracy: 0.3416 - val_loss: 1.6746 - val_accuracy: 0.5227
Epoch 2/12
469/469 [==============================] - 213s 454ms/step - loss: 1.1844 - accuracy: 0.6353 - val_loss: 1.2752 - val_accuracy: 0.6165
Epoch 3/12
469/469 [==============================] - 211s 450ms/step - loss: 0.9188 - accuracy: 0.7169 - val_loss: 1.1081 - val_accuracy: 0.6604
Epoch 4/12
469/469 [==============================] - 213s 454ms/step - loss: 0.8054 - accuracy: 0.7503 - val_loss: 1.0051 - val_accuracy: 0.6873
Epoch 5/12
469/469 [==============================] - 212s 452ms/step - loss: 0.7404 - accuracy: 0.7710 - val_loss: 0.9493 - val_accuracy: 0.7028
Epoch 6/12
469/469 [==============================] - 212s 451ms/step - loss: 0.6963 - accuracy: 0.7845 - val_loss: 0.8988 - val_accuracy: 0.7176
Epoch 7/12
469/469 [==============================] - 213s 453ms/step - loss: 0.6649 - accuracy: 0.7959 - val_loss: 0.8592 -

In [16]:
# optimizer = Nadam

# model_2_4: change of optimizer to Nadam and compile

model_2_4 = tf.keras.Sequential()
model_2_4.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model_2_4.add(Conv2D(64, (3, 3), activation='relu'))
model_2_4.add(MaxPooling2D(pool_size=(2, 2)))
model_2_4.add(Dropout(0.25))
model_2_4.add(tf.keras.layers.Flatten())
model_2_4.add(tf.keras.layers.Dense(128, activation='relu'))
model_2_4.add(Dropout(0.5))
model_2_4.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

model_2_4.compile(loss='sparse_categorical_crossentropy',
              optimizer='Nadam',
              metrics=['accuracy'])

In [17]:
model_2_4.fit(x_train, y_train,
           batch_size=batch_size,
           epochs=epochs,
           verbose=1,
           validation_data=(x_test, y_test))

Epoch 1/12
469/469 [==============================] - 97s 206ms/step - loss: 0.4006 - accuracy: 0.8765 - val_loss: 0.3753 - val_accuracy: 0.8869
Epoch 2/12
469/469 [==============================] - 95s 203ms/step - loss: 0.1686 - accuracy: 0.9496 - val_loss: 0.2696 - val_accuracy: 0.9219
Epoch 3/12
469/469 [==============================] - 95s 204ms/step - loss: 0.1271 - accuracy: 0.9604 - val_loss: 0.2380 - val_accuracy: 0.9340
Epoch 4/12
469/469 [==============================] - 96s 204ms/step - loss: 0.0979 - accuracy: 0.9690 - val_loss: 0.2271 - val_accuracy: 0.9407
Epoch 5/12
469/469 [==============================] - 96s 204ms/step - loss: 0.0836 - accuracy: 0.9736 - val_loss: 0.2184 - val_accuracy: 0.9427
Epoch 6/12
469/469 [==============================] - 96s 204ms/step - loss: 0.0676 - accuracy: 0.9787 - val_loss: 0.1975 - val_accuracy: 0.9490
Epoch 7/12
469/469 [==============================] - 96s 204ms/step - loss: 0.0602 - accuracy: 0.9804 - val_loss: 0.2283 - val_ac

#### This is the first dataset in .npz format

In [15]:
!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-imgs.npz?raw=True
!mv kmnist-train-imgs.npz?raw=True kmnist-train-imgs.npz
!ls -lahr kmnist-train-imgs.npz

!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-labels.npz?raw=True
!mv kmnist-train-labels.npz?raw=True kmnist-train-labels.npz
!ls -lahr kmnist-train-labels.npz

!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-test-imgs.npz?raw=True
!mv kmnist-test-imgs.npz?raw=True kmnist-test-imgs.npz
!ls -lahr kmnist-test-imgs.npz

!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-test-labels.npz?raw=True
!mv kmnist-test-labels.npz?raw=True kmnist-test-labels.npz
!ls -lahr kmnist-test-labels.npz

--2021-06-11 17:21:46--  http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-imgs.npz?raw=True
Resolving codh.rois.ac.jp (codh.rois.ac.jp)... 136.187.88.58
Connecting to codh.rois.ac.jp (codh.rois.ac.jp)|136.187.88.58|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18384171 (18M)
Saving to: ‘kmnist-train-imgs.npz?raw=True’

kmnist-train-imgs.n 100%[===================>]  17.53M  4.25MB/s    in 4.9s    

2021-06-11 17:21:52 (3.61 MB/s) - ‘kmnist-train-imgs.npz?raw=True’ saved [18384171/18384171]

-rw-rw-r-- 1 spark spark 18M Feb  4  2019 kmnist-train-imgs.npz
--2021-06-11 17:21:54--  http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-labels.npz?raw=True
Resolving codh.rois.ac.jp (codh.rois.ac.jp)... 136.187.88.58
Connecting to codh.rois.ac.jp (codh.rois.ac.jp)|136.187.88.58|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29700 (29K)
Saving to: ‘kmnist-train-labels.npz?raw=True’

kmnist-train-labels 100%[===================>]

### Model:  K Nearest Neighbors

In [16]:
def load(f):
    return np.load(f)['arr_0']

In [17]:
x_train = load('kmnist-train-imgs.npz')
y_train = load('kmnist-train-labels.npz')
x_test = load('kmnist-test-imgs.npz')
y_test = load('kmnist-test-labels.npz')

In [18]:
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

In [19]:
model_3 = KNeighborsClassifier(n_neighbors=4, weights='distance', n_jobs=-1)
print('Fit', model_3)
model_3.fit(x_train, y_train)

Fit KNeighborsClassifier(n_jobs=-1, n_neighbors=4, weights='distance')


KNeighborsClassifier(n_jobs=-1, n_neighbors=4, weights='distance')

In [20]:
print('Evaluate', model_3)
score = model_3.score(x_test, y_test)
print('The accuracy is:', score)

Evaluate KNeighborsClassifier(n_jobs=-1, n_neighbors=4, weights='distance')
The accuracy is: 0.921


### Model: K Nearest Neighbors with Principal Component Analysis dimensionality reduction

In [ ]:
# KNN with dim reduction:  the code is currently broken, skip this for now

pca = PCA(n_components=60, random_state=0)

In [ ]:
x_train = pca.fit_transform(x_train)
x_test = pca.transform

In [ ]:
model_4 = KNeighborsClassifier(n_neighbors=4, weights='distance', n_jobs=-1)
print('Fit', model_4)
model_4.fit(x_train, y_train)

In [ ]:
print('Evaluate', model_4)
score = model_4.score(x_test, y_test)
print('The accuracy is:', score)

## Second dataset

#### 232,365 images, 49 classes, each image is 28 x 28 or represented by a 794 element array

In [17]:
# download the train and test labels and images, 
# which are available onlyh in .npz format 

!wget http://codh.rois.ac.jp/kmnist/dataset/k49/k49-train-imgs.npz?raw=True
!mv k49-train-imgs.npz?raw=True k49-train-imgs.npz
!ls -lahr k49-train-imgs.npz

!wget http://codh.rois.ac.jp/kmnist/dataset/k49/k49-train-labels.npz?raw=True
!mv k49-train-labels.npz?raw=True k49-train-labels.npz
!ls -lahr k49-train-labels.npz

!wget http://codh.rois.ac.jp/kmnist/dataset/k49/k49-test-imgs.npz?raw=True
!mv k49-test-imgs.npz?raw=True k49-test-imgs.npz
!ls -lahr k49-test-imgs.npz

!wget http://codh.rois.ac.jp/kmnist/dataset/k49/k49-test-labels.npz?raw=True
!mv k49-test-labels.npz?raw=True k49-test-labels.npz
!ls -lahr k49-test-labels.npz

--2021-06-21 21:33:38--  http://codh.rois.ac.jp/kmnist/dataset/k49/k49-train-imgs.npz?raw=True
Resolving codh.rois.ac.jp (codh.rois.ac.jp)... 136.187.88.58
Connecting to codh.rois.ac.jp (codh.rois.ac.jp)|136.187.88.58|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66117696 (63M)
Saving to: ‘k49-train-imgs.npz?raw=True’

k49-train-imgs.npz? 100%[===================>]  63.05M  8.11MB/s    in 8.6s    

2021-06-21 21:33:47 (7.32 MB/s) - ‘k49-train-imgs.npz?raw=True’ saved [66117696/66117696]

-rw-rw-r-- 1 spark spark 64M Feb  4  2019 k49-train-imgs.npz
--2021-06-21 21:33:49--  http://codh.rois.ac.jp/kmnist/dataset/k49/k49-train-labels.npz?raw=True
Resolving codh.rois.ac.jp (codh.rois.ac.jp)... 136.187.88.58
Connecting to codh.rois.ac.jp (codh.rois.ac.jp)|136.187.88.58|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 164485 (161K)
Saving to: ‘k49-train-labels.npz?raw=True’

k49-train-labels.np 100%[===================>] 160.63K   383KB/s    

In [18]:
def load(f):
    return np.load(f)['arr_0']

In [19]:
x_train_k49 = load('k49-train-imgs.npz')
y_train_k49 = load('k49-train-labels.npz')
x_test_k49 = load('k49-test-imgs.npz')
y_test_k49 = load('k49-test-labels.npz')

x_train_k49 = x_train_k49.reshape(-1, 784)
x_test_k49 = x_test_k49.reshape(-1, 784)


# the numpy arrays used in the keras models need to be between 0 and 1
# we keep x_train_k49 and x_test_k49 in the format of between 0 and 255
# for use in the K-nearest Neighbors model below

train_images_k49 = x_train_k49 / 255
test_images_k49 = x_test_k49 / 255

# and the labels can be the same

train_labels_k49 = y_train_k49
test_labels_k49 = y_test_k49

In [9]:
# model_1_k49 def
# we have to change the last layer due to the 49 categories

model_1_k49 = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(49, activation=tf.nn.softmax)
])

In [10]:
# model_1_k49 compile
model_1_k49.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
# model_1_k49 fit
# as in the first data set, this model could probably use more epochs
model_1_k49.fit(train_images_k49, train_labels_k49, epochs=5, verbose=1, validation_data=(test_images_k49, test_labels_k49))

Epoch 1/5
7262/7262 [==============================] - 14s 2ms/step - loss: 0.9142 - accuracy: 0.7662 - val_loss: 1.0799 - val_accuracy: 0.7278
Epoch 2/5
7262/7262 [==============================] - 13s 2ms/step - loss: 0.5710 - accuracy: 0.8474 - val_loss: 0.9618 - val_accuracy: 0.7581
Epoch 3/5
7262/7262 [==============================] - 13s 2ms/step - loss: 0.4910 - accuracy: 0.8670 - val_loss: 0.9151 - val_accuracy: 0.7734
Epoch 4/5
7262/7262 [==============================] - 13s 2ms/step - loss: 0.4495 - accuracy: 0.8786 - val_loss: 0.9089 - val_accuracy: 0.7800
Epoch 5/5
7262/7262 [==============================] - 13s 2ms/step - loss: 0.4226 - accuracy: 0.8843 - val_loss: 0.9129 - val_accuracy: 0.7817


In [9]:
# model_1_2_k49 def

model_1_2_k49 = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),    
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),    
    tf.keras.layers.Dense(49, activation=tf.nn.softmax)
])

In [10]:
# model_1_2_k49 compile
model_1_2_k49.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
# model_1_2_k49 fit
model_1_2_k49.fit(train_images_k49, train_labels_k49, epochs=20, verbose=1, validation_data=(test_images_k49, test_labels_k49))

Epoch 1/20
7262/7262 [==============================] - 17s 2ms/step - loss: 0.8359 - accuracy: 0.7735 - val_loss: 0.9507 - val_accuracy: 0.7440
Epoch 2/20
7262/7262 [==============================] - 16s 2ms/step - loss: 0.5021 - accuracy: 0.8600 - val_loss: 0.8355 - val_accuracy: 0.7785
Epoch 3/20
7262/7262 [==============================] - 16s 2ms/step - loss: 0.4239 - accuracy: 0.8810 - val_loss: 0.7762 - val_accuracy: 0.7992
Epoch 4/20
7262/7262 [==============================] - 16s 2ms/step - loss: 0.3812 - accuracy: 0.8923 - val_loss: 0.7687 - val_accuracy: 0.8008
Epoch 5/20
7262/7262 [==============================] - 16s 2ms/step - loss: 0.3527 - accuracy: 0.8997 - val_loss: 0.7548 - val_accuracy: 0.8121
Epoch 6/20
7262/7262 [==============================] - 17s 2ms/step - loss: 0.3314 - accuracy: 0.9058 - val_loss: 0.7723 - val_accuracy: 0.8113
Epoch 7/20
7262/7262 [==============================] - 16s 2ms/step - loss: 0.3148 - accuracy: 0.9100 - val_loss: 0.7624 - val_ac

In [21]:
model_1_2_k49.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_4 (Dense)              (None, 49)                6321      
Total params: 156,337
Trainable params: 156,337
Non-trainable params: 0
_________________________________________________________________


In [9]:
# model_1_5_k49 def

model_1_5_k49 = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),    
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),    
    tf.keras.layers.Dense(49, activation=tf.nn.softmax)
])

In [10]:
# model_1_5_k49 compile

model_1_5_k49.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
# model_1_5_k49 fit

model_1_5_k49.fit(train_images_k49, train_labels_k49, epochs=20, verbose=1, validation_data=(test_images_k49, test_labels_k49))

Epoch 1/20
7262/7262 [==============================] - 15s 2ms/step - loss: 1.7135 - accuracy: 0.5690 - val_loss: 1.5775 - val_accuracy: 0.6086
Epoch 2/20
7262/7262 [==============================] - 14s 2ms/step - loss: 0.8724 - accuracy: 0.7705 - val_loss: 1.2330 - val_accuracy: 0.6849
Epoch 3/20
7262/7262 [==============================] - 14s 2ms/step - loss: 0.6760 - accuracy: 0.8188 - val_loss: 1.0666 - val_accuracy: 0.7251
Epoch 4/20
7262/7262 [==============================] - 14s 2ms/step - loss: 0.5714 - accuracy: 0.8449 - val_loss: 0.9663 - val_accuracy: 0.7480
Epoch 5/20
7262/7262 [==============================] - 14s 2ms/step - loss: 0.5047 - accuracy: 0.8612 - val_loss: 0.8925 - val_accuracy: 0.7698
Epoch 6/20
7262/7262 [==============================] - 14s 2ms/step - loss: 0.4553 - accuracy: 0.8747 - val_loss: 0.8496 - val_accuracy: 0.7793
Epoch 7/20
7262/7262 [==============================] - 14s 2ms/step - loss: 0.4175 - accuracy: 0.8841 - val_loss: 0.8400 - val_ac

In [12]:
# model_1_5_k49 fit

model_1_5_k49.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_4 (Dense)              (None, 49)                6321      
Total params: 156,337
Trainable params: 156,337
Non-trainable params: 0
_________________________________________________________________


In [12]:
# model_2_k49 def

batch_size = 128
num_classes = 49       #  here we specify the number of classes as 49 instead of 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

x_train = train_images_k49
x_test = test_images_k49
y_train = train_labels_k49
y_test = test_labels_k49

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

model_2_k49 = tf.keras.Sequential()
model_2_k49.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model_2_k49.add(Conv2D(64, (3, 3), activation='relu'))
model_2_k49.add(MaxPooling2D(pool_size=(2, 2)))
model_2_k49.add(Dropout(0.25))
model_2_k49.add(tf.keras.layers.Flatten())
model_2_k49.add(tf.keras.layers.Dense(128, activation='relu'))
model_2_k49.add(Dropout(0.5))
model_2_k49.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

# model_2 compile

model_2_k49.compile(optimizer='adam', 
          loss='sparse_categorical_crossentropy',
          metrics=['accuracy'])

In [13]:
# fit model2_k49

model_2_k49.fit(x_train, y_train,
         batch_size=batch_size,
         epochs=epochs,
         verbose=1,
         validation_data=(x_test, y_test))

Epoch 1/12
1816/1816 [==============================] - 342s 188ms/step - loss: 0.9383 - accuracy: 0.7530 - val_loss: 0.6622 - val_accuracy: 0.8259
Epoch 2/12
1816/1816 [==============================] - 340s 187ms/step - loss: 0.5146 - accuracy: 0.8584 - val_loss: 0.5186 - val_accuracy: 0.8644
Epoch 3/12
1816/1816 [==============================] - 338s 186ms/step - loss: 0.4178 - accuracy: 0.8831 - val_loss: 0.4378 - val_accuracy: 0.8841
Epoch 4/12
1816/1816 [==============================] - 338s 186ms/step - loss: 0.3619 - accuracy: 0.8981 - val_loss: 0.3942 - val_accuracy: 0.8978
Epoch 5/12
1816/1816 [==============================] - 336s 185ms/step - loss: 0.3248 - accuracy: 0.9068 - val_loss: 0.3849 - val_accuracy: 0.9000
Epoch 6/12
1816/1816 [==============================] - 336s 185ms/step - loss: 0.2997 - accuracy: 0.9139 - val_loss: 0.3704 - val_accuracy: 0.9067
Epoch 7/12
1816/1816 [==============================] - 335s 185ms/step - loss: 0.2788 - accuracy: 0.9189 - val_

In [20]:
# model_2_1_k49 def

batch_size = 128
num_classes = 49       #  here we specify the number of classes as 49 instead of 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

x_train = train_images_k49
x_test = test_images_k49
y_train = train_labels_k49
y_test = test_labels_k49

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

model_2_1_k49 = tf.keras.Sequential()
model_2_1_k49.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model_2_1_k49.add(Conv2D(64, (3, 3), activation='relu'))
model_2_1_k49.add(MaxPooling2D(pool_size=(2, 2)))
model_2_1_k49.add(Dropout(0.25))
model_2_1_k49.add(tf.keras.layers.Flatten())
model_2_1_k49.add(tf.keras.layers.Dense(128, activation='relu'))
model_2_1_k49.add(Dropout(0.5))
model_2_1_k49.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

In [21]:
# change of optimizer and compile model_2_k49

model_2_1_k49.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adadelta(learning_rate=1.0),
              metrics=['accuracy'])

In [ ]:
# fit model_2_k49 after change of optmizer

model_2_1_k49.fit(x_train, y_train,
           batch_size=batch_size,
           epochs=epochs,
           verbose=1,
           validation_data=(x_test, y_test))

### Model:  K Nearest Neighbors - k49

In [16]:
# recall that the two numpy image arrays used below, namely x_train_k49 and x_test_k49
# are arrays of integers, which works fine for KNN

In [17]:
model_3_k49 = KNeighborsClassifier(n_neighbors=4, weights='distance', n_jobs=-1)
print('Fit', model_3_k49)
model_3_k49.fit(x_train_k49, y_train_k49)

Fit KNeighborsClassifier(n_jobs=-1, n_neighbors=4, weights='distance')


KNeighborsClassifier(n_jobs=-1, n_neighbors=4, weights='distance')

In [ ]:
print('Evaluate', model_3_k49)
score = model_3_k49.score(x_test_k49, y_test_k49)
print('The accuracy is:', score)

Evaluate KNeighborsClassifier(n_jobs=-1, n_neighbors=4, weights='distance')
